In [76]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
RANDOM_STATE = 42
HORIZON_DAYS = 3  # горизонт прогноза 

In [77]:
data_dir = Path('.')
fires_raw = pd.read_csv(data_dir / 'fires.csv')
supplies_raw = pd.read_csv(data_dir / 'supplies.csv')
temp_raw = pd.read_csv(data_dir / 'temperature.csv')
weather_files = sorted(data_dir.glob('weather_data_*.csv'))
weather_raw = pd.concat([pd.read_csv(f) for f in weather_files], ignore_index=True)

print('fires_raw.shape:', fires_raw.shape)
print('supplies_raw.shape:', supplies_raw.shape)
print('temp_raw.shape:', temp_raw.shape)
print('weather_raw.shape:', weather_raw.shape)
print("Пропуски в исходных таблицах")
print('fires_raw:\n', fires_raw.isna().sum())
print('supplies_raw:\n', supplies_raw.isna().sum())
print('temp_raw:\n', temp_raw.isna().sum())
print('weather_raw:\n', weather_raw.isna().sum())

fires_raw.shape: (486, 8)
supplies_raw.shape: (6323, 7)
temp_raw.shape: (4106, 7)
weather_raw.shape: (61344, 11)
Пропуски в исходных таблицах
fires_raw:
 Дата составления    0
Груз                0
Вес по акту, тн     0
Склад               0
Дата начала         0
Дата оконч.         0
Нач.форм.штабеля    0
Штабель             0
dtype: int64
supplies_raw:
 ВыгрузкаНаСклад    0
Наим. ЕТСНГ        0
Штабель            0
ПогрузкаНаСудно    0
На склад, тн       0
На судно, тн       2
Склад              0
dtype: int64
temp_raw:
 Склад                         0
Штабель                       0
Марка                         0
Максимальная температура      0
Пикет                       459
Дата акта                     0
Смена                       157
dtype: int64
weather_raw:
 date                 0
t                    0
p                    0
humidity             0
precipitation        0
wind_dir             0
v_avg                0
v_max                0
cloudcover           0
visibility   

In [78]:
def preprocess_weather(df):
    """Дневная агрегация погодных данных"""
    df = df.copy()
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    agg = df.groupby('date').agg({
        't': 'mean',
        'p': 'mean',
        'humidity': 'mean',
        'precipitation': 'sum',
        'v_avg': 'mean',
        'v_max': 'mean',
        'cloudcover': 'mean',
        'weather_code': 'max'
    }).reset_index()

    agg.rename(columns={
        't': 't_mean',
        'p': 'p_mean',
        'humidity': 'humidity_mean',
        'precipitation': 'precip_sum',
        'v_avg': 'wind_avg',
        'v_max': 'wind_max',
        'cloudcover': 'cloudcover_mean',
        'weather_code': 'weather_code_max'
    }, inplace=True)

    num_cols = agg.select_dtypes(include=[np.number]).columns
    agg[num_cols] = agg[num_cols].fillna(agg[num_cols].median())
    agg['date'] = pd.to_datetime(agg['date'])
    return agg

In [79]:
def preprocess_supplies(df):
    """Дневные погрузки и выгрузки по складу/штабелю."""
    df = df.copy()
    df['ВыгрузкаНаСклад'] = pd.to_datetime(df['ВыгрузкаНаСклад'])
    df['ПогрузкаНаСудно'] = pd.to_datetime(df['ПогрузкаНаСудно'])

    # На склад (прибытие)
    in_df = df.groupby(['Склад', 'Штабель', df['ВыгрузкаНаСклад'].dt.date])['На склад, тн'].sum().reset_index()
    in_df.rename(columns={'ВыгрузкаНаСклад': 'date', 'На склад, тн': 'in_tons'}, inplace=True)
    in_df.rename(columns={in_df.columns[2]: 'date'}, inplace=True)
    # На судно (убытие)
    out_df = df.groupby(['Склад', 'Штабель', df['ПогрузкаНаСудно'].dt.date])['На судно, тн'].sum().reset_index()
    out_df.rename(columns={'ПогрузкаНаСудно': 'date', 'На судно, тн': 'out_tons'}, inplace=True)
    out_df.rename(columns={out_df.columns[2]: 'date'}, inplace=True)

    base = pd.merge(in_df, out_df, on=['Склад', 'Штабель', 'date'], how='outer')
    base['in_tons'] = base['in_tons'].fillna(0)
    base['out_tons'] = base['out_tons'].fillna(0)
    base['net_tons'] = base['in_tons'] - base['out_tons']
    base['date'] = pd.to_datetime(base['date'])
    return base

In [80]:
def preprocess_temperature(df):
    """Максимум температуры по дню и штабелю."""
    df = df.copy()
    df['Дата акта'] = pd.to_datetime(df['Дата акта'])
    df['date'] = df['Дата акта'].dt.date

    tmp = df.groupby(['Склад', 'Штабель', 'date'])['Максимальная температура'].max().reset_index()
    tmp.rename(columns={'Максимальная температура': 'max_temp_stack'}, inplace=True)
    tmp['date'] = pd.to_datetime(tmp['date'])
    return tmp

In [81]:
def preprocess_fires(df):
    """Дата возгорания (метка) и дата формирования штабеля."""
    df = df.copy()
    df['Дата начала'] = pd.to_datetime(df['Дата начала'])
    df['Дата оконч.'] = pd.to_datetime(df['Дата оконч.'])
    df['Нач.форм.штабеля'] = pd.to_datetime(df['Нач.форм.штабеля'])

    fires_daily = df.groupby(['Склад', 'Штабель', df['Дата начала'].dt.date]).size().reset_index(name='fire')
    fires_daily['date'] = pd.to_datetime(fires_daily['Дата начала'])
    fires_daily.drop(columns=['Дата начала'], inplace=True)
    fires_daily['fire'] = 1

    formation = df.groupby(['Склад', 'Штабель'])['Нач.форм.штабеля'].min().reset_index()
    formation.rename(columns={'Нач.форм.штабеля': 'formation_date'}, inplace=True)
    return fires_daily, formation

In [82]:
weather_daily = preprocess_weather(weather_raw)
supplies_daily = preprocess_supplies(supplies_raw)
temp_daily = preprocess_temperature(temp_raw)
fires_daily, formation_df = preprocess_fires(fires_raw)

print('\nweather_daily:\n', weather_daily.head(1))
print('supplies_daily:\n', supplies_daily.head(1))
print('temp_daily:\n', temp_daily.head(1))
print('fires_daily:\n', fires_daily.head(1))
print('formation_df:\n', formation_df.head(1))


weather_daily:
         date    t_mean       p_mean  humidity_mean  precip_sum   wind_avg  \
0 2015-01-01 -2.083333  1025.679167      72.958333         0.0  24.495833   

    wind_max  cloudcover_mean  weather_code_max  
0  40.558333           70.625                 3  
supplies_daily:
    Склад  Штабель       date     in_tons  out_tons    net_tons
0      3        1 2019-01-02  11984.1925       0.0  11984.1925
temp_daily:
    Склад  Штабель       date  max_temp_stack
0      3        0 2020-05-29            26.3
fires_daily:
    Склад  Штабель  fire       date
0      3        1     1 2019-08-04
formation_df:
    Склад  Штабель formation_date
0      3        1     2019-01-01


In [83]:
# Создаем общий датафрейм признаков
base = pd.merge(supplies_daily, temp_daily, on=['Склад', 'Штабель', 'date'], how='outer')
base = pd.merge(base, weather_daily, on='date', how='left')
base = pd.merge(base, formation_df, on=['Склад', 'Штабель'], how='left')
base = pd.merge(base, fires_daily[['Склад', 'Штабель', 'date', 'fire']], on=['Склад', 'Штабель', 'date'], how='left')
base['fire'] = base['fire'].fillna(0).astype(int)

# возраст штабеля
base['age_days'] = (base['date'] - base['formation_date']).dt.days
base['age_days'] = base['age_days'].fillna(0)

# обработка пропусков
num_cols = ['in_tons', 'out_tons', 'net_tons', 'max_temp_stack',
            't_mean', 'p_mean', 'humidity_mean', 'precip_sum',
            'wind_avg', 'wind_max', 'cloudcover_mean', 'weather_code_max', 'age_days']

for col in num_cols:
    if col in base.columns:
        base[col] = base[col].fillna(0)

base = base.dropna(subset=['Склад', 'Штабель', 'date'])
base['Склад'] = base['Склад'].astype(int)
base['Штабель'] = base['Штабель'].astype(int)

print('Итоговый датафрейм base')
print(base.head())
print('\nbase shape:', base.shape)
print('\nРаспределение fire:')
print(base['fire'].value_counts())

Итоговый датафрейм base
   Склад  Штабель       date     in_tons  out_tons    net_tons  \
0      3        0 2020-05-29      0.0000       0.0      0.0000   
1      3        1 2019-01-02  11984.1925       0.0  11984.1925   
2      3        1 2019-01-06  11427.7060       0.0  11427.7060   
3      3        1 2019-01-07  11984.1925       0.0  11984.1925   
4      3        1 2019-01-10  25899.2620       0.0  25899.2620   

   max_temp_stack     t_mean       p_mean  humidity_mean  precip_sum  \
0            26.3  15.566667  1011.179167      84.666667        10.8   
1             0.0   7.529167  1013.716667      75.541667         5.2   
2             0.0   3.558333  1006.454167      89.583333         3.9   
3             0.0   1.816667  1009.112500      85.000000        14.5   
4             0.0   7.579167  1017.166667      63.375000         5.2   

    wind_avg   wind_max  cloudcover_mean  weather_code_max formation_date  \
0  18.950000  28.408333        82.083333              61.0           

In [84]:
FEATURE_COLS_TO_FORECAST = ['max_temp_stack', 'in_tons', 'out_tons', 'net_tons', 't_mean', 'p_mean', 'humidity_mean', 'precip_sum', 'wind_avg', 'wind_max', 'cloudcover_mean', 'age_days']

def train_feature_models(df, feature_cols = FEATURE_COLS_TO_FORECAST, horizon_days = HORIZON_DAYS):
    """Обучаем модели для предсказания признаки(t + d) для d = 1...horizon_days."""
    models = {}
    df = df.copy()
    df = df.sort_values(['Склад', 'Штабель', 'date'])
    group = df.groupby(['Склад', 'Штабель'])
    
    for d in range(1, horizon_days + 1):
        shifted = df.copy()
        for col in feature_cols:
            shifted[f'{col}_target'] = group[col].shift(-d)
        target_cols = [f'{c}_target' for c in feature_cols]
        train_rows = shifted.dropna(subset=target_cols)
        if train_rows.empty:
            continue
        X = train_rows[feature_cols]
        Y = train_rows[target_cols]
        model = MultiOutputRegressor(RandomForestRegressor(
                n_estimators=100,
                random_state=RANDOM_STATE))
        model.fit(X, Y)
        models[d] = model
        print(f'Обучена модель прогнозирования признаков для горизонта d={d}')
    return models

In [85]:
def forecast_features_for_horizon(feature_models, base_features, feature_cols=FEATURE_COLS_TO_FORECAST,horizon_days=HORIZON_DAYS):
    """На вход: признаки на день запроса прогноза (по штабелям), на выход: предсказанные признаки на каждый день горизонта."""
    results = []
    for d in range(1, horizon_days + 1):
        model = feature_models.get(d)
        X = base_features[feature_cols]
        Y_pred = model.predict(X)
        pred_df = base_features[['Склад', 'Штабель', 'date']].copy()
        for i, col in enumerate(feature_cols):
            pred_df[col] = Y_pred[:, i]
        pred_df['delta_day'] = d
        pred_df['target_date'] = pred_df['date'] + pd.to_timedelta(d, unit='D')
        results.append(pred_df)
    if results:
        all_preds = pd.concat(results, ignore_index=True)
    else:
        all_preds = pd.DataFrame()
    return all_preds

In [86]:
CLASS_FEATURES = ['Склад', 'Штабель'] + FEATURE_COLS_TO_FORECAST
df_model = base.dropna(subset=CLASS_FEATURES + ['fire']).copy()
train_idx, test_idx = train_test_split(
    df_model.index,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=df_model['fire'])
X_train = df_model.loc[train_idx, CLASS_FEATURES]
y_train = df_model.loc[train_idx, 'fire']

X_test  = df_model.loc[test_idx, CLASS_FEATURES]
y_test  = df_model.loc[test_idx, 'fire']

base_train = base.loc[train_idx]    
base_test  = base.loc[test_idx] 

feature_models_train = train_feature_models(
    base_train,
    feature_cols=FEATURE_COLS_TO_FORECAST,
    horizon_days=HORIZON_DAYS
)


Обучена модель прогнозирования признаков для горизонта d=1
Обучена модель прогнозирования признаков для горизонта d=2
Обучена модель прогнозирования признаков для горизонта d=3


In [ ]:
clf = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
clf.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [88]:
def compute_fire_event_metrics(df, window_days = 2):
    """Precision/Recall/F1 с допуском ±window_days по дате."""
    true_events = df[df['fire_true'] == 1][['Склад', 'Штабель', 'date']].drop_duplicates().reset_index(drop=True)
    pred_events = df[df['fire_pred'] == 1][['Склад', 'Штабель', 'date']].drop_duplicates().reset_index(drop=True)

    true_list = list(true_events.itertuples(index=False, name=None))
    pred_list = list(pred_events.itertuples(index=False, name=None))

    matched_true = set()
    matched_pred = set()

    for i, (sk_t, stack_t, date_t) in enumerate(true_list):
        for j, (sk_p, stack_p, date_p) in enumerate(pred_list):
            if j in matched_pred:
                continue
            if (sk_t == sk_p) and (stack_t == stack_p):
                if abs((date_p - date_t).days) <= window_days:
                    matched_true.add(i)
                    matched_pred.add(j)
                    break

    TP_fire = len(matched_true)
    FN_fire = len(true_list) - TP_fire
    FP_fire = len(pred_list) - TP_fire

    precision_fire = TP_fire / (TP_fire + FP_fire) if (TP_fire + FP_fire) > 0 else 0.0
    recall_fire = TP_fire / (TP_fire + FN_fire) if (TP_fire + FN_fire) > 0 else 0.0
    if precision_fire + recall_fire > 0:
        f1_fire = 2 * precision_fire * recall_fire / (precision_fire + recall_fire)
    else:
        f1_fire = 0.0

    return {
        'TP_fire': TP_fire,
        'FP_fire': FP_fire,
        'FN_fire': FN_fire,
        'precision_fire': precision_fire,
        'recall_fire': recall_fire,
        'f1_fire': f1_fire,
        'true_events': len(true_list),
        'pred_events': len(pred_list)
    }

In [89]:
def evaluate_pipeline_with_forecasted_features(
    df_base: pd.DataFrame,
    feature_models: dict,
    clf,
    horizon_days: int = HORIZON_DAYS,
    proba_threshold: float = 0.5,
    window_days: int = 2
):
    """
    Оценивает качество классификации, когда
    в классификатор подаются не реальные признаки, а предсказанные
    моделями из feature_models на горизонте 1..horizon_days.

    df_base  – исходный датафрейм base (с колонками 'Склад', 'Штабель', 'date', 'fire' и фичами)
    feature_models – словарь {d: модель}, как вернул train_feature_models
    clf      – обученный классификатор
    proba_threshold – порог вероятности
    """

    df = df_base.copy().sort_values(['Склад', 'Штабель', 'date'])
    df['date'] = pd.to_datetime(df['date'])
    group = df.groupby(['Склад', 'Штабель'])
    all_rows = []
    for d in range(1, horizon_days + 1):
        model = feature_models.get(d)
        df_shift = df.copy()
        df_shift['fire_future'] = group['fire'].shift(-d)

        valid = df_shift.dropna(subset=['fire_future']).copy()
        valid['fire_future'] = valid['fire_future'].astype(int)

        # 1) Базовые признаки на день t
        X_feat = valid[FEATURE_COLS_TO_FORECAST]

        # 2) Прогноз фич на t + d
        Y_pred_features = model.predict(X_feat)

        # 3) Формируем вход для классификатора: ['Склад', 'Штабель'] + предсказанные фичи
        X_cls = valid[['Склад', 'Штабель']].copy()
        for i, col in enumerate(FEATURE_COLS_TO_FORECAST):
            X_cls[col] = Y_pred_features[:, i]

        # 4) Прогноз пожара
        proba = clf.predict_proba(X_cls)[:, 1]
        pred = (proba >= proba_threshold).astype(int)

        # 5) Собираем в единый датафрейм: метки относятся к дате t + d
        tmp = valid[['Склад', 'Штабель', 'date', 'fire_future']].copy()
        tmp.rename(columns={'fire_future': 'fire_true'}, inplace=True)
        tmp['target_date'] = tmp['date'] + pd.to_timedelta(d, unit='D')
        tmp['delta_day'] = d
        tmp['fire_pred'] = pred
        tmp['proba'] = proba

        all_rows.append(tmp)

    eval_df = pd.concat(all_rows, ignore_index=True)
    print('Размер eval_df (по всем горизонтам):', eval_df.shape)
    day_df = eval_df.copy()
    day_df['date'] = day_df['target_date']
    
    fire_df = day_df[['Склад', 'Штабель', 'date', 'fire_true', 'fire_pred']].copy()

    fire_metrics_forecast = compute_fire_event_metrics(
        fire_df,
        window_days=window_days
    )

    print(f'метрики (±{window_days} дней) на предсказанных признаках')
    for k, v in fire_metrics_forecast.items():
        print(f'{k}: {v:.3f}')

    return eval_df, fire_metrics_forecast

In [117]:
eval_df_forecast, fire_metrics_forecast = evaluate_pipeline_with_forecasted_features(
    df_base=base_test,
    feature_models=feature_models_train,
    clf=clf,
    horizon_days=HORIZON_DAYS,
    proba_threshold=0.5,
    window_days=2
)

Размер eval_df (по всем горизонтам): (3051, 8)
метрики (±2 дней) на предсказанных признаках
TP_fire: 87.000
FP_fire: 48.000
FN_fire: 56.000
precision_fire: 0.644
recall_fire: 0.608
f1_fire: 0.626
true_events: 143.000
pred_events: 135.000
